<a href="https://colab.research.google.com/github/Crazydodo123/code-ml24/blob/main/autogluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install --upgrade pip
!python -m pip install autogluon

In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [15]:
df = TabularDataset("new_branded_data.csv")
df = df.dropna(subset=["choice"])
df = df.drop(['id', 'Unnamed: 0','flight_departure_datetime', 'purchase_datetime', 'ticket_id'], axis=1)
df = df.dropna()
df

Loaded data from: new_branded_data.csv | Columns = 21 / 21 | Rows = 256665 -> 256665


,od,time_diff,trip_type,branded_fare,number_of_pax,ADVS_price,PREF_price,ADVS_capacity,PREF_capacity,ADVS_inventory,PREF_inventory,pref_inv_full,advs_ratio,pref_ratio,total_cap,choice
2,BBBDDD,13035420.0,0,1,2,35,80,50,10,50,7,1,1.000,0.7,60,pref
3,BBBDDD,12129540.0,0,3,1,0,0,50,10,50,5,1,1.000,0.5,60,nochoice
4,BBBDDD,11508960.0,0,3,1,0,0,50,10,50,5,1,1.000,0.5,60,pref
7,BBBDDD,9089040.0,1,3,1,0,0,50,10,48,4,1,0.960,0.4,60,pref
8,BBBDDD,8474820.0,1,0,3,44,81,50,10,48,3,1,0.960,0.3,60,nochoice
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256657,AAABBB,239400.0,0,0,3,41,59,250,50,196,0,0,0.784,0.0,300,nochoice
256658,AAABBB,224880.0,0,0,3,56,69,250,50,196,0,0,0.784,0.0,300,nochoice
256660,AAABBB,206220.0,1,2,1,0,69,250,50,192,0,0,0.768,0.0,300,advs
256661,AAABBB,191700.0,0,3,2,0,0,250,50,191,0,0,0.764,0.0,300,advs


In [16]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
label = 'choice'
train_data[label].describe()

In [ ]:
predictor = TabularPredictor(label=label, eval_metric="f1_weighted").fit(train_data)

In [ ]:
leaderboard = predictor.leaderboard(test_data)
leaderboard

In [40]:
best_model_name = predictor.model_best
best_model = predictor._trainer.load_model(best_model_name)

In [43]:
import pandas as pd
df_to_predict = pd.read_csv("new_branded_baseline.csv")

columns_to_drop = ['id',
                   'Unnamed: 0',
          'ticket_id',
          'choice',
          'flight_departure_datetime',
          'purchase_datetime']
X_to_predict = df_to_predict.drop(columns_to_drop, axis=1)
X_to_predict = X_to_predict.dropna()

In [50]:
predictions = predictor.predict(X_to_predict)
predictions

In [47]:
df_to_predict['choice'] = predictions
df_to_predict[['id', 'choice']].to_csv('autogluon_predictions.csv', index=False)